In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Function to download stock price data
def get_stock_data(symbol, start_date, end_date):
    data = yf.download(symbol, start=start_date, end=end_date)
    return data

# Function to create sequences from time series data
def create_sequences(data, sequence_length):
    sequences = []
    for i in range(len(data) - sequence_length):
        sequence = data[i : i + sequence_length]
        sequences.append(sequence)
    return np.array(sequences)

# Function to split data into features and target
def split_data(sequences):
    X = sequences[:, :-1]
    y = sequences[:, -1]
    return X, y

# Function to build LSTM model
def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(64, input_shape=input_shape, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Load stock price data
symbol = '005930.KS'  # Samsung Electronics stock symbol on Yahoo Finance
start_date = '2020-01-01'
end_date = '2023-07-21'
stock_data = get_stock_data(symbol, start_date, end_date)

# Preprocess data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(stock_data['Close'].values.reshape(-1, 1))

# Create sequences and split data into features and target
sequence_length = 30  # You can adjust this value based on your preference
sequences = create_sequences(scaled_data, sequence_length)
X, y = split_data(sequences)

# Reshape input data for LSTM (samples, timesteps, features)
X = X.reshape(X.shape[0], X.shape[1], 1)

# Build LSTM model
model = build_lstm_model(input_shape=(X.shape[1], 1))

# Train the model
model.fit(X, y, epochs=100, batch_size=32, verbose=1)

# Predict stock price one week later
last_sequence = scaled_data[-sequence_length:]
last_sequence = last_sequence.reshape(1, last_sequence.shape[0], 1)
predicted_scaled_price = model.predict(last_sequence)
predicted_price = scaler.inverse_transform(predicted_scaled_price)[0][0]

print("Predicted stock price one week later:", predicted_price)


[*********************100%***********************]  1 of 1 completed
Epoch 1/100
27/27 [==============================] - 6s 18ms/step - loss: 0.0895
Epoch 2/100
27/27 [==============================] - 0s 17ms/step - loss: 0.0048
Epoch 3/100
27/27 [==============================] - 1s 22ms/step - loss: 0.0028
Epoch 4/100
27/27 [==============================] - 1s 19ms/step - loss: 0.0023
Epoch 5/100
27/27 [==============================] - 1s 30ms/step - loss: 0.0022
Epoch 6/100
27/27 [==============================] - 0s 18ms/step - loss: 0.0022
Epoch 7/100
27/27 [==============================] - 1s 20ms/step - loss: 0.0019
Epoch 8/100
27/27 [==============================] - 0s 18ms/step - loss: 0.0018
Epoch 9/100
27/27 [==============================] - 1s 19ms/step - loss: 0.0017
Epoch 10/100
27/27 [==============================] - 0s 17ms/step - loss: 0.0016
Epoch 11/100
27/27 [==============================] - 0s 18ms/step - loss: 0.0016
Epoch 12/100
27/27 [==================

ValueError: ignored